# Задание:
## Написать скрипт python / jupyter ноутбук, который выведет пары городов с населением больше 50 тыс.человек и расстоянием меньше 150 км.

In [61]:
from tqdm import tqdm
import requests
import pandas as pd
import numpy as np
import os.path
import overpy
import re

Для получения информации о городах использую https://www.geonames.org/.

Загрузка данных через https://public.opendatasoft.com API

In [2]:
if os.path.isfile('geonames-all-cities-with-a-population-1000.csv'):
    pass
else:
    url = ('https://public.opendatasoft.com/api/records/1.0/download/?dataset=geonames-all-cities-with-a-population-1000&'
           + 'fields=ascii_name,latitude,longitude,population,country')
    response = requests.get(url, stream=True)

    # Progress bar для загрузки файла
    total_size_in_bytes= int(response.headers.get('content-length', 0))
    block_size = 1024
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)

    with open('geonames-all-cities-with-a-population-1000.csv', 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()

In [16]:
data = pd.read_csv('geonames-all-cities-with-a-population-1000.csv', sep = ';')

In [4]:
data

,ascii_name,latitude,longitude,population,country
0,Brejo Santo,-7.49333,-38.98722,27384,Brazil
1,Arari,-3.45361,-44.78000,16777,Brazil
2,Alhandra,-7.43861,-34.91444,10788,Brazil
3,Umbauba,-11.38333,-37.65778,12880,Brazil
4,Teutonia,-29.44806,-51.80639,21834,Brazil
...,...,...,...,...,...
137491,Barnesville,33.05457,-84.15575,6625,United States
137492,Canton,34.23676,-84.49076,25469,United States
137493,Clarkston,33.80955,-84.23964,12215,United States
137494,Cordele,31.96351,-83.78239,10943,United States


Отбор городов с населением больше 50000 человек.

In [17]:
all_cities = data[data.population > 50000].reset_index(drop=True)

In [6]:
all_cities

,ascii_name,latitude,longitude,population,country
0,Sapiranga,-29.63806,-51.00694,76051,Brazil
1,Nossa Senhora do Socorro,-10.85500,-37.12611,163993,Brazil
2,Japeri,-22.64306,-43.65333,95101,Brazil
3,Itanhaem,-24.18306,-46.78889,90385,Brazil
4,Gravatai,-29.94218,-50.99278,238778,Brazil
...,...,...,...,...,...
9050,Kalush,49.02398,24.37206,66406,Ukraine
9051,Boryspil',50.35269,30.95501,55000,Ukraine
9052,Delray Beach,26.46146,-80.07282,66255,United States
9053,Weston,26.10037,-80.39977,69959,United States


Функция для вычисления расстояния между городами.

In [9]:
def broadcasting_based_lat_lng_elementwise(data1, data2): 
    # data1, data2 are the data arrays with 2 cols and they hold
    # lat., lng. values in those cols respectively
    
    # перевод из градусов в радианы
    data1 = np.deg2rad(data1)                     
    data2 = np.deg2rad(data2)                     

    lat1 = data1[:,0]                     
    lng1 = data1[:,1]         

    lat2 = data2[:,0]                     
    lng2 = data2[:,1]         

    diff_lat = lat1 - lat2
    diff_lng = lng1 - lng2
    
    d = np.sin(diff_lat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(diff_lng/2)**2
    
    return np.round(2 * 6371 * np.arcsin(np.sqrt(d)), 1)

Функция для формирования итоговой таблицы, возвращает словарь, ключи которого — столбцы таблицы.

In [10]:
def make_city_pairs(data, columns):
    
    result_dict = {k:[] for k in columns}
    
    # проходим по таблице и сравниваем выбранный город с городами, которые находятся ниже в таблице
    for first_city in data[:-1].itertuples():
        second_cities = data[first_city.Index + 1:]
        
        # формируем массивы необходимой размерности
        first_city_arr = np.array([[first_city.latitude, first_city.longitude]])
        second_city_arr = np.array(tuple(zip(second_cities.latitude, second_cities.longitude)))
        
        distance = broadcasting_based_lat_lng_elementwise(first_city_arr, second_city_arr)
        
        # фильтруем полученные расстояния и таблицу с остальными городами
        second_cities = second_cities[distance < 150]
        distance = distance[distance < 150]
        
        # создаем дополнительные колонки
        second_cities['distance'] = distance
        second_cities['city1'] = first_city.name
        second_cities['coordinates1'] = f'({first_city.latitude}, {first_city.longitude})'
        second_cities['population1'] = first_city.population
        
        second_cities['coordinates2'] = tuple(zip(second_cities.latitude, second_cities.longitude))
        second_cities.rename(columns={'name': 'city2', 'population': 'population2'}, inplace=True)
        
        # сохраняем каждую колонку в ключ словаря с соответсвующим названием
        [result_dict[column].extend(list(second_cities[column])) for column in columns]
        
    return result_dict

In [18]:
%%time
columns = ['city1', 'coordinates1', 'city2', 'coordinates2', 'distance', 'population1', 'population2']

all_cities.rename(columns={'ascii_name': 'name'}, inplace=True)

answer_dict = make_city_pairs(all_cities, columns)

Wall time: 43.8 s


В итоговый результат было решено добавить координаты городов, для того, чтобы различать города с одинаковым названием.

In [19]:
answer = pd.DataFrame(answer_dict)

- `city1`, `city2` — названия городов
- `coordinates1`, `coordinates2` — координаты городов (широта, долгота)
- `population1`, `population2` — население городов

In [70]:
answer

,city1,coordinates1,city2,coordinates2,distance,population1,population2
0,Sapiranga,"(-29.63806, -51.00694)",Gravatai,"(-29.94218, -50.992779999999996)",33.8,76051,238778
1,Sapiranga,"(-29.63806, -51.00694)",Guaiba,"(-30.11389, -51.325)",61.2,76051,101024
2,Sapiranga,"(-29.63806, -51.00694)",Montenegro,"(-29.688609999999997, -51.46111)",44.2,76051,54057
3,Sapiranga,"(-29.63806, -51.00694)",Lajeado,"(-29.46694, -51.96139)",94.3,76051,65407
4,Sapiranga,"(-29.63806, -51.00694)",Farroupilha,"(-29.225, -51.34778)",56.6,76051,57650
...,...,...,...,...,...,...,...
124921,Pollachi,"(10.65825, 77.0085)",Kottayam,"(9.58692, 76.52131999999999)",130.5,88214,59437
124922,Erzurum,"(39.908609999999996, 41.27694)",Bingol,"(38.88472, 40.49389)",132.2,420691,80568
124923,UEnye,"(41.13139, 37.2825)",Carsamba,"(41.198890000000006, 36.721940000000004)",47.5,77585,50459
124924,Kavakli,"(41.09258, 28.33172)",Corlu,"(41.15917, 27.8)",45.1,50502,202578


In [481]:
answer.to_csv('answer.csv')

Второй источник — https://www.openstreetmap.org/

Получение информации о городах через Overpass API

In [24]:
api = overpy.Overpass()
result1 = api.query("""node["place"="city"];out body;""")

In [22]:
result2 = api.query("""node["place"="town"];out body;""")

In [91]:
len(result1.nodes), len(result2.nodes)

(10085, 101781)

Обработка полученного ответа.

In [94]:
results = []
results.extend(result1.nodes)
results.extend(result2.nodes)
data_dict = {'name': [], 'latitude': [], 'longitude': [], 'population': []}

for node in results:
    # проверяем, чтобы в результате было название и население города
    if 'population' in node.tags and 'name' in node.tags:
        # убираем лишние символы в населении
        pop = node.tags['population'].replace(',', '').replace('\'', '').replace('.', '').replace('~', '')
        pop = pop.replace('\u202f', '').replace('\u200e','').replace('+', '').replace('±', '').replace(' ', '')
        res = re.match('\d+', pop)
        if res:
            pop = res.group(0)
            if 'name:en' in node.tags:
                data_dict['name'].append(node.tags['name:en'])
            else:
                data_dict['name'].append(node.tags['name'])
                
            data_dict['latitude'].append(float(node.lat))
            data_dict['longitude'].append(float(node.lon))
            data_dict['population'].append(int(pop))

Сохранение полученной таблицы.

In [81]:
data_OSM = pd.DataFrame(data_dict)

if os.path.isfile('data_OSM.csv'):
    pass
else: 
    data_OSM.to_csv('data_OSM.csv')

In [82]:
data_OSM = pd.read_csv('data_OSM.csv', index_col=0)

In [96]:
data_OSM

,name,latitude,longitude,population
0,Novosibirsk,55.028217,82.923451,1625600
1,London,51.507322,-0.127647,8416535
2,Wellington,-41.288795,174.777211,201900
3,Itajaí,-26.911788,-48.666985,205271
4,Wells,51.209451,-2.645120,11000
...,...,...,...,...
43848,Dungripali,21.044483,83.552171,5000
43849,Kashechewan First Nation,52.293460,-81.639364,2537
43850,Redford,42.400780,-83.307806,48362
43851,Devon Meadows,-38.170702,145.307621,1548


Аналогичная обработка.

In [97]:
all_cities_OSM = data_OSM[data_OSM.population > 50000].reset_index(drop=True)

In [98]:
all_cities_OSM

,name,latitude,longitude,population
0,Novosibirsk,55.028217,82.923451,1625600
1,London,51.507322,-0.127647,8416535
2,Wellington,-41.288795,174.777211,201900
3,Itajaí,-26.911788,-48.666985,205271
4,Leeds,53.797418,-1.543794,766399
...,...,...,...,...
8634,Hashtrud,37.475894,47.049962,57199
8635,大沥镇,23.112105,113.104890,610000
8636,Orland Park,41.630663,-87.853629,57634
8637,Diepsloot,-25.930950,28.010590,138329


In [99]:
%%time
columns = ['city1', 'coordinates1', 'city2', 'coordinates2', 'distance', 'population1', 'population2']
all_cities_OSM.rename(columns={'lat': 'latitude', 'lon': 'longitude'}, inplace=True)
answer_dict_OSM = make_city_pairs(all_cities_OSM, columns)

Wall time: 40.9 s


In [100]:
answer_OSM = pd.DataFrame(answer_dict_OSM)

In [101]:
answer_OSM

,city1,coordinates1,city2,coordinates2,distance,population1,population2
0,Novosibirsk,"(55.0282171, 82.9234509)",Berdsk,"(54.75795, 83.1068095)",32.3,1625600,104237
1,Novosibirsk,"(55.0282171, 82.9234509)",Yurga,"(55.7136, 84.934799)",148.2,1625600,81073
2,Novosibirsk,"(55.0282171, 82.9234509)",Iskitim,"(54.6426269, 83.3035676)",49.3,1625600,56411
3,London,"(51.5073219, -0.1276474)",Leicester,"(52.6361398, -1.1330789)",143.1,8416535,305700
4,London,"(51.5073219, -0.1276474)",Oxford,"(51.7520131, -1.2578499)",82.6,8416535,165000
...,...,...,...,...,...,...,...
132348,La,"(5.56619, -0.1604294)",Adenta,"(5.7041391, -0.1687965)",15.4,98683,96478
132349,La,"(5.56619, -0.1604294)",Lashibi,"(5.6850117, -0.0383551)",18.9,98683,78539
132350,Arimba,"(-14.9481082, 13.5951829)",Huíla,"(-15.0521838, 13.5443138)",12.8,55000,55000
132351,Zhuhe,"(21.7709598, 108.3494894)",Wenchang,"(21.7381688, 108.3597636)",3.8,57712,61700


In [87]:
answer_OSM[(answer_OSM.city1 == 'Novosibirsk') | (answer_OSM.city2 == 'Novosibirsk')]

,city1,coordinates1,city2,coordinates2,distance,population1,population2
0,Novosibirsk,"(55.0282171, 82.9234509)",Berdsk,"(54.75795, 83.1068095)",32.3,1625600,104237
1,Novosibirsk,"(55.0282171, 82.9234509)",Yurga,"(55.7136, 84.934799)",148.2,1625600,81073
2,Novosibirsk,"(55.0282171, 82.9234509)",Iskitim,"(54.642626899999996, 83.3035676)",49.3,1625600,56411


In [89]:
answer_OSM.to_csv('answer_OSM.csv')